In [ ]:
!pip install kagglehub pandas numpy scikit-learn xgboost lightgbm catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.4 MB/s eta 0:00:00


In [ ]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("ealtman2019/ibm-transactions-for-anti-money-laundering-aml")

print("Dataset downloaded to:", path)


100%|██████████| 7.42G/7.42G [01:13<00:00, 109MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml/versions/7


HI (High Illicit Ratio):More laundering transactions

In [ ]:
import pandas as pd

# Define the dataset path
dataset_path = "/root/.cache/kagglehub/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml/versions/7/HI-Small_Trans.csv"

# Load the dataset
df = pd.read_csv(dataset_path)

# Display the first few rows
df.head()


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [ ]:
print(df.columns)


Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Convert 'Timestamp' to datetime and extract useful features
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["Year"] = df["Timestamp"].dt.year
df["Month"] = df["Timestamp"].dt.month
df["Day"] = df["Timestamp"].dt.day
df["Hour"] = df["Timestamp"].dt.hour

# Drop non-useful columns
df.drop(columns=["Timestamp", "Account", "Account.1"], inplace=True)

# Encode categorical features
label_enc = LabelEncoder()
for col in ["From Bank", "To Bank", "Receiving Currency", "Payment Currency", "Payment Format"]:
    df[col] = label_enc.fit_transform(df[col])

# Define features and target variable
X = df.drop(columns=["Is Laundering"])  # Features
y = df["Is Laundering"]  # Target (1 = laundering, 0 = legitimate)

# Split into train and test sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data preprocessing and split completed successfully!")


Data preprocessing and split completed successfully!


Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Train Decision Tree
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)

# Predict
y_pred_dt = dt_model.predict(X_test)

# Evaluate
print("Decision Tree Performance:")
print(classification_report(y_test, y_pred_dt))


Decision Tree Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1014582
           1       0.69      0.13      0.21      1087

    accuracy                           1.00   1015669
   macro avg       0.84      0.56      0.61   1015669
weighted avg       1.00      1.00      1.00   1015669



XG Boost

In [ ]:
from xgboost import XGBClassifier

# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate
print("XGBoost Performance:")
print(classification_report(y_test, y_pred_xgb))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:56:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1014582
           1       0.92      0.15      0.26      1087

    accuracy                           1.00   1015669
   macro avg       0.96      0.58      0.63   1015669
weighted avg       1.00      1.00      1.00   1015669



LG Boost

In [ ]:
from lightgbm import LGBMClassifier

# Train LightGBM
lgb_model = LGBMClassifier()
lgb_model.fit(X_train, y_train)

# Predict
y_pred_lgb = lgb_model.predict(X_test)

# Evaluate
print("LightGBM Performance:")
print(classification_report(y_test, y_pred_lgb))


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4090, number of negative: 4058586
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.505922 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1102
[LightGBM] [Info] Number of data points in the train set: 4062676, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001007 -> initscore=-6.900045
[LightGBM] [Info] Start training from score -6.900045
LightGBM Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1014582
           1       0.38      0.16      0.22      1087

    accuracy                           1.00   1015669
   macro avg       0.69      0.58      0.61   1015669
weighted avg       1.00      1.00      1.00   1015669



CAT Boost

In [ ]:
from catboost import CatBoostClassifier

# Train CatBoost
cat_model = CatBoostClassifier(verbose=0)
cat_model.fit(X_train, y_train)

# Predict
y_pred_cat = cat_model.predict(X_test)

# Evaluate
print("CatBoost Performance:")
print(classification_report(y_test, y_pred_cat))


CatBoost Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1014582
           1       0.90      0.20      0.33      1087

    accuracy                           1.00   1015669
   macro avg       0.95      0.60      0.67   1015669
weighted avg       1.00      1.00      1.00   1015669



In [ ]:
from sklearn.metrics import accuracy_score

# Compute Accuracy for Each Model
dt_acc = accuracy_score(y_test, y_pred_dt)
xgb_acc = accuracy_score(y_test, y_pred_xgb)
lgb_acc = accuracy_score(y_test, y_pred_lgb)
cat_acc = accuracy_score(y_test, y_pred_cat)

print(f"Decision Tree Accuracy: {dt_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"LightGBM Accuracy: {lgb_acc:.4f}")
print(f"CatBoost Accuracy: {cat_acc:.4f}")


Decision Tree Accuracy: 0.9990
XGBoost Accuracy: 0.9991
LightGBM Accuracy: 0.9988
CatBoost Accuracy: 0.9991


In [ ]:
import joblib

# Save the best performing model
best_model = cat_model  # Replace with the best model based on evaluation
joblib.dump(best_model, "best_aml_model.pkl")

print("Best model saved successfully!")


Best model saved successfully!
